<a href="https://colab.research.google.com/github/nodeblackbox/Soundprocessing/blob/main/equalizer_eq_and_treble_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Install necessary libraries
!pip install librosa ipywidgets scipy soundfile

# # Enable ipywidgets for Colab
!jupyter nbextension enable --py widgetsnbextension


import numpy as np
import librosa
import soundfile as sf
import scipy.signal as signal
from IPython.display import Audio, display, clear_output
import ipywidgets as widgets
from io import BytesIO

# File upload widget
upload = widgets.FileUpload(
    accept='.wav, .mp3',
    multiple=False,
    description='Upload Audio File'
)

# Sliders for audio parameters
bass_slider = widgets.FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.5, description='Bass (dB)')
mids_slider = widgets.FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.5, description='Mids (dB)')
treble_slider = widgets.FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.5, description='Treble (dB)')
compression_slider = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Compression')
noise_reduction_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Noise Reduction')

# Button to apply changes
apply_button = widgets.Button(
    description='Apply Changes and Play',
    button_style='success',
    icon='play'
)

# Output widget
output = widgets.Output()

def load_audio(uploaded_file):
    """Load audio from uploaded file."""
    if uploaded_file and len(uploaded_file) > 0:
        # Get the first uploaded file
        file_info = next(iter(uploaded_file.values()))
        content = file_info['content']
        audio_bytes = BytesIO(content)
        try:
            y, sr = librosa.load(audio_bytes, sr=None, mono=True)
            return y, sr
        except Exception as e:
            print(f"Error loading audio: {str(e)}")
            return None, None
    return None, None

def apply_eq(y, sr, bass_gain, mids_gain, treble_gain):
    """Apply equalization to the audio signal."""
    nyquist = 0.5 * sr

    def design_filter(freq_range, gain):
        low, high = freq_range[0] / nyquist, freq_range[1] / nyquist
        if low >= 1.0: low = 0.99
        if high >= 1.0: high = 0.99
        b, a = signal.butter(2, [low, high], btype='bandpass')
        return b, a, gain

    # Process each frequency band
    bass_b, bass_a, _ = design_filter((20, 250), bass_gain)
    mids_b, mids_a, _ = design_filter((250, 4000), mids_gain)
    treble_b, treble_a, _ = design_filter((4000, 20000), treble_gain)

    y_bass = signal.lfilter(bass_b, bass_a, y) * (10 ** (bass_gain / 20))
    y_mids = signal.lfilter(mids_b, mids_a, y) * (10 ** (mids_gain / 20))
    y_treble = signal.lfilter(treble_b, treble_a, y) * (10 ** (treble_gain / 20))

    return y_bass + y_mids + y_treble

def apply_compression(y, threshold=0.5, ratio=2.0):
    """Apply compression to the audio signal."""
    y_compressed = np.copy(y)
    over_threshold = np.abs(y) > threshold
    y_compressed[over_threshold] = np.sign(y[over_threshold]) * (threshold + (np.abs(y[over_threshold]) - threshold) / ratio)
    return y_compressed

def apply_noise_reduction(y, sr, reduction_level=0.5):
    """Apply noise reduction to the audio signal."""
    # Use a shorter noise sample to avoid dimension mismatch
    noise_sample_length = min(int(0.1 * sr), len(y))
    noise_clip = y[:noise_sample_length]

    # Compute noise profile
    noise_power = np.mean(noise_clip**2)
    signal_power = np.mean(y**2)
    snr = 10 * np.log10(signal_power / noise_power + 1e-6)

    # Apply noise reduction
    reduction = reduction_level * snr
    noise_profile = np.tile(noise_clip, len(y) // len(noise_clip) + 1)[:len(y)]
    y_reduced = y - reduction * noise_profile

    return y_reduced

def process_and_play(y, sr, bass, mids, treble, compression, noise_reduction):
    """Process the audio with all effects and play the result."""
    try:
        # Apply Equalization
        y_eq = apply_eq(y, sr, bass, mids, treble)

        # Apply Compression
        y_compressed = apply_compression(y_eq, threshold=0.5, ratio=compression)

        # Apply Noise Reduction
        if noise_reduction > 0.0:
            y_processed = apply_noise_reduction(y_compressed, sr, reduction_level=noise_reduction)
        else:
            y_processed = y_compressed

        # Normalize the audio
        max_val = np.max(np.abs(y_processed))
        if max_val > 0:
            y_processed = y_processed / max_val

        # Convert to 16-bit PCM
        y_int16 = np.int16(y_processed * 32767)

        # Save and play
        buffer = BytesIO()
        sf.write(buffer, y_int16, sr, format='WAV')
        buffer.seek(0)

        display(Audio(buffer.read(), rate=sr))

    except Exception as e:
        print(f"Error processing audio: {str(e)}")

def on_apply_button_clicked(b):
    """Handle button click event."""
    with output:
        clear_output()
        y, sr = load_audio(upload.value)
        if y is not None:
            print("Processing audio...")
            bass = bass_slider.value
            mids = mids_slider.value
            treble = treble_slider.value
            compression = compression_slider.value
            noise_reduction = noise_reduction_slider.value

            process_and_play(y, sr, bass, mids, treble, compression, noise_reduction)
        else:
            print("Please upload an audio file first.")

# Connect button to click handler
apply_button.on_click(on_apply_button_clicked)

# Create main UI layout
main_layout = widgets.VBox([
    widgets.HTML("<h2>Audio Processor</h2>"),
    upload,
    widgets.HTML("<h3>Audio Parameters</h3>"),
    widgets.VBox([bass_slider, mids_slider, treble_slider, compression_slider, noise_reduction_slider]),
    apply_button,
    output
])

# Display the application
display(main_layout)
print("Audio processor ready. Upload a file and adjust parameters, then click 'Apply Changes and Play'.")